In [1]:
# Import necessary PySpark libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import findspark
findspark.init()

In [2]:
# Initialize Spark session
spark = SparkSession.builder.master("local[*]").appName("FeatureEngineeringAndModeling").getOrCreate()

24/11/05 17:27:45 WARN Utils: Your hostname, Sarabjots-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.216.21.221 instead (on interface en0)
24/11/05 17:27:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/05 17:27:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/05 17:27:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Load the data into a Spark DataFrame
df = spark.read.csv("/Users/sarabjotsingh/Downloads/Feature_engineering.csv", header=True, inferSchema=True)

In [4]:
df = df.select([col(c).alias(c.replace(" ", "_")) for c in df.columns])  # Rename columns to remove spaces

In [5]:
# Define a window specification based on the 'open' column as ordering
window = Window.orderBy("open")
# List of features for which we want to create lagged columns
selected_features = ['up', 'prev_diff', 'daydiff']

In [6]:
# Define the lag values to be created
lags = [1, 2, 3]
# Generate Lagged Features
for feature in selected_features:
    for lag_value in lags:
        lagged_col_name = f"{feature}_lag{lag_value}"
        df = df.withColumn(lagged_col_name, lag(col(feature), lag_value).over(window))
df = df.dropna()  # Drop rows with null values introduced by lagging

In [7]:
# Split the data into training and testing sets 
train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

In [8]:
# Save training and test DataFrames to CSV files
train_df.write.csv("/Users/sarabjotsingh/Downloads/train_data.csv", header=True, mode='overwrite')
test_df.write.csv("/Users/sarabjotsingh/Downloads/test_data.csv", header=True, mode='overwrite')

24/11/05 17:27:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:27:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:27:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:27:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/11/05 17:27:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:27:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance

In [9]:
# Exclude non-numeric columns for VectorAssembler
numeric_columns = [col for col, dtype in df.dtypes if dtype in ['double', 'int'] and col != 'RSI']

# Assemble the features into a single vector column for both train and test sets
assembler = VectorAssembler(inputCols=numeric_columns, outputCol="features")
train_df = assembler.transform(train_df).select("features", "RSI")
test_df = assembler.transform(test_df).select("features", "RSI")


In [10]:
# Standardize the feature vector
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(train_df)
train_df = scaler_model.transform(train_df).select("scaled_features", "RSI").withColumnRenamed("scaled_features", "features")
test_df = scaler_model.transform(test_df).select("scaled_features", "RSI").withColumnRenamed("scaled_features", "features")

24/11/05 17:27:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:27:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:27:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:27:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [ ]:
# Save the model to a specified path
model_save_path = "/Users/sarabjotsingh/Downloads/scaler_model"  # Specify the path for saving the model
scaler_model.save(model_save_path)

print(f"Model saved at: {model_save_path}")

In [12]:
# Initialize and train the Random Forest Regressor
rf = RandomForestRegressor(labelCol="RSI", featuresCol="features")
model = rf.fit(train_df)

24/11/05 17:27:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:27:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:27:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:28:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:28:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [13]:
# Test set evaluation
predictions = model.transform(test_df)
evaluator = RegressionEvaluator(labelCol="RSI", predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)
print(f"Test Mean Squared Error (MSE): {mse}")

24/11/05 17:28:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:28:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:28:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:28:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/05 17:28:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Test Mean Squared Error (MSE): 273.35097142599477


In [14]:
# Get feature importances as a list of tuples (column name, importance value)
feature_importances_list = [(numeric_columns[i], float(importance)) for i, importance in enumerate(model.featureImportances)]

# Create DataFrame from the list of feature importances
importances_df = spark.createDataFrame(feature_importances_list, ["Feature", "Importance"]) \
    .orderBy(col("Importance"), ascending=False)

# Show feature importances
importances_df.show()


+------------------+--------------------+
|           Feature|          Importance|
+------------------+--------------------+
|                up|  0.4125928767726175|
|                dn|  0.3303327668259434|
|     capital_gains| 0.07338511782933646|
|              MACD| 0.06462853075584815|
|         prev_diff| 0.06160036300550298|
|           daydiff|0.012540378836400246|
| rolling_dividends|0.012278439508470029|
|        SignalLine|0.009430165388200754|
|             close|0.007826710843842549|
|         adj_close|0.006963291976127055|
|             EMA12|0.002572990354764...|
|               low|0.001411980650843712|
|        prev_close|0.001357822267814372|
|              high|0.001164360241819...|
|previous_day_close|7.628574270410179E-4|
|              open|7.032836995436566E-4|
|            volume|1.618630266070473E-4|
|             EMA26|1.383189019135208...|
|           maxdiff|8.960368542244482E-5|
|         dividends|4.810862036690144...|
+------------------+--------------

In [15]:
# Save the model to a specified path
model_save_path = "/Users/sarabjotsingh/Downloads/rf_model_1"  # Specify the path for saving the model
model.save(model_save_path)

print(f"Model saved at: {model_save_path}")

24/11/05 20:12:42 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 427854 ms exceeds timeout 120000 ms
24/11/05 20:12:42 WARN SparkContext: Killing executors is not supported by current scheduler.
24/11/05 20:12:43 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$